In [1]:
import os

## Input parameters
- Specify scaffold inputs for RFdiffusion (with/without backbone threading), target pdb and hotspots to target
- Define number of RFdiffusion docking we want to perform and number of ProteinMPNN sequences to sample per diffusion - total designs = RFdockings * mpnn_per_design
- Other parameters

In [13]:
array_number=14 # If sequence scaffolds, then it should product of number 7
array_limit=21

scaffold_dir="../../scaffolds/sequence_scaffolds/" #if it contains multiple subdirectories, it gets threaded...
separate_scaffold_folders=True
thread_binders=True #if True, poly-GLY binders will be threaded and relaxed before ProteinMPNN

prefix="tlr4-ago1"
target_pdb="examples/TLR4-short.pdb"
#output="output/$prefix" # output gets created by rfdiffusion
hotspots='ppi.hotspot_res=[A324,A325,A327,A329,A344,A347,A349,A351]'

num_of_diffusions=10 # Number of RF diffusions per script
total_mpnn=100 # Total mpnn sequences for per design
mpnn_per_design=100 # Filtered mpnn sequences for AF2

# Other parameters
num_recycles=9 # AF2 recycles
sampling_temp=0.15 # ProteinMPNN sampling temperature
print(f"This will generate {num_of_diffusions*mpnn_per_design} designs per array job!")

This will generate 1000 designs per array job!


In [14]:
def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subdirectories.append(entry.path)
    return subdirectories

if separate_scaffold_folders:
    scaffold_list=get_subdirectories(scaffold_dir)
    scaffolds=len(scaffold_list)
    print(f"Script will run {scaffolds} array scripts for each scaffold with {array_number//scaffolds} jobs ({num_of_diffusions*mpnn_per_design} designs each) with total of {(array_number/scaffolds)*(num_of_diffusions*mpnn_per_design)} sampled designes per scaffold")

Script will run 7 array scripts for each scaffold with 2 jobs (1000 designs each) with total of 2000.0 sampled designes per scaffold


In [15]:
scaffold_list

['../../scaffolds/sequence_scaffolds/2H4E_19',
 '../../scaffolds/sequence_scaffolds/3H_3',
 '../../scaffolds/sequence_scaffolds/3H_12',
 '../../scaffolds/sequence_scaffolds/3H_8',
 '../../scaffolds/sequence_scaffolds/3H_6',
 '../../scaffolds/sequence_scaffolds/4H_9',
 '../../scaffolds/sequence_scaffolds/lcb3']

## Runs slurm array script

In [16]:
import subprocess

if separate_scaffold_folders:
    array_number=array_number//scaffolds
    array_limit=array_limit//scaffolds

    for i in range(scaffolds):
        # Setup arguments
        scaffold_dir=scaffold_list[i]
        script_arguments=f"{num_of_diffusions} {total_mpnn} {mpnn_per_design} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
        bash_arguments=f"--array=1-{array_number}%{array_limit}"
        
        command = f"sbatch {bash_arguments} helper_scripts/binder_dock.sh {script_arguments}"

        # Run the array bash script
        subprocess.run(command, shell=True)

else:
    script_arguments=f"{num_of_diffusions} {total_mpnn} {mpnn_per_design} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {hotspots}"
    bash_arguments=f"--array=1-{array_number}%{array_limit}"
        
    command = f"sbatch {bash_arguments} helper_scripts/binder_dock.sh {script_arguments}"

    # Run the array bash script
    subprocess.run(command, shell=True)




#print(f"command: {command}")

Submitted batch job 144642
Submitted batch job 144644
Submitted batch job 144645
Submitted batch job 144646
Submitted batch job 144647
Submitted batch job 144648
Submitted batch job 144649


In [19]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          144642_1       gpu binder_d  tsatler  R       0:05      1 compute-0-11
          144642_2       gpu binder_d  tsatler  R       0:05      1 compute-0-11
          144644_1       gpu binder_d  tsatler  R       0:03      1 compute-0-11
          144644_2       gpu binder_d  tsatler  R       0:03      1 compute-0-11
          144645_1       gpu binder_d  tsatler  R       0:03      1 compute-0-11
          144645_2       gpu binder_d  tsatler  R       0:03      1 compute-0-11
          144646_1       gpu binder_d  tsatler  R       0:03      1 compute-0-11
          144646_2       gpu binder_d  tsatler  R       0:03      1 compute-0-12
          144647_1       gpu binder_d  tsatler  R       0:03      1 compute-0-12
          144647_2       gpu binder_d  tsatler  R       0:03      1 compute-0-12
          144648_1       gpu binder_d  tsatler  R       0:03      1 compute-0-12
          144648_2      